In [1]:
# Ensemble Learning
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
# Read the CSV and Perform Basic Data Cleaning
# Load the data
file_path = Path('LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
# Split the Data into Training and Testing
# Create our features
x_features = df.drop(columns='loan_status')
new_x_features = pd.get_dummies(x_features)
X = new_x_features

# Create our target
y = df["loan_status"].values.reshape(-1, 1)

In [6]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [7]:
# Check the balance of our target values
df["loan_status"].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [8]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
X_train.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,51612.000000,51612.000000,51612.000000,5.161200e+04,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,...,51612.000000,51612.0,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.0,51612.0
mean,16689.106700,0.127642,480.676883,8.863669e+04,21.786729,0.218438,0.495389,12.577947,0.125475,17566.003623,...,0.178292,1.0,0.123634,0.876366,0.383031,0.616969,0.859451,0.140549,1.0,1.0
std,10266.626708,0.048183,287.489861,1.275942e+05,20.227410,0.721159,0.756235,6.007003,0.336195,21828.912860,...,0.382762,0.0,0.329167,0.329167,0.486130,0.486130,0.347559,0.347559,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9175.000000,0.088100,265.730000,5.000000e+04,13.880000,0.000000,0.000000,8.000000,0.000000,6295.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.790000,7.291250e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12056.500000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,647.800000,1.038735e+05,26.630000,0.000000,1.000000,16.000000,0.000000,21721.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [10]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Ensemble Learners
# In this section, we will compare two ensemble algorithms to determine which algorithm results in the best performance. (cont'd) 
# We will train a Balanced Random Forest Classifier and an Easy Ensemble classifier .

# Balanced Random Forest Classifier
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [14]:
# Calculated the balanced accuracy score
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.778784371911514

In [15]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   62,    30],
       [ 1991, 15122]])

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.67      0.88      0.06      0.77      0.58        92
   low_risk       1.00      0.88      0.67      0.94      0.77      0.61     17113

avg / total       0.99      0.88      0.68      0.93      0.77      0.61     17205



In [17]:
# List the features sorted in descending order by feature importance
importances = brf.feature_importances_
sorted_indices = np.argsort(importances)[::-1]
print(*X_train.columns[sorted_indices], sep = "\n")

total_rec_prncp
total_pymnt_inv
total_rec_int
last_pymnt_amnt
total_pymnt
int_rate
issue_d_Jan-2019
mo_sin_old_rev_tl_op
dti
max_bal_bc
revol_bal
avg_cur_bal
total_rev_hi_lim
mths_since_recent_inq
total_bc_limit
total_bal_il
out_prncp_inv
bc_util
mths_since_rcnt_il
mo_sin_old_il_acct
tot_hi_cred_lim
installment
annual_inc
total_il_high_credit_limit
bc_open_to_buy
out_prncp
tot_cur_bal
mths_since_recent_bc
loan_amnt
all_util
total_bal_ex_mort
il_util
num_sats
total_acc
num_il_tl
num_rev_accts
open_acc
mo_sin_rcnt_rev_tl_op
num_op_rev_tl
issue_d_Mar-2019
inq_last_12m
num_bc_tl
num_actv_rev_tl
num_rev_tl_bal_gt_0
num_bc_sats
mo_sin_rcnt_tl
inq_last_6mths
acc_open_past_24mths
pct_tl_nvr_dlq
inq_fi
percent_bc_gt_75
mort_acc
open_act_il
num_tl_op_past_12m
open_rv_24m
total_cu_tl
total_rec_late_fee
open_il_24m
num_actv_bc_tl
issue_d_Feb-2019
next_pymnt_d_May-2019
delinq_2yrs
open_rv_12m
open_acc_6m
open_il_12m
next_pymnt_d_Apr-2019
verification_status_Not Verified
num_accts_ever_120_pd
tot_co

In [18]:
# Easy Ensemble Classifier
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(random_state=42)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(random_state=42)

In [19]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9292916140538976

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   86,     6],
       [ 1304, 15809]])

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.06      0.93      0.92      0.12      0.93      0.86        92
   low_risk       1.00      0.92      0.93      0.96      0.93      0.86     17113

avg / total       0.99      0.92      0.93      0.96      0.93      0.86     17205



In [22]:
# Final Questions - 4 Questions

# Question 1: Which model had the best balanced accuracy score?
# Answer: The Easy Ensemble Classifier had the best balanced accuracy score at 0.9292916140538976.

In [23]:
# Question 2: Which model had the best recall score?
# Answer: The Easy Ensemble Classifier had the best recall score at 0.93 for high_risk, 0.92 for low_risk, and 0.92 for avg/total.

In [24]:
# Question 3: Which model had the best geometric mean score?
# Answer: The Easy Ensemble Classifier had the best geometric score at 0.93 for high_risk, low_risk, and avg/total.

In [25]:
# Question 4: What are the top three features?
# Answer: The top 3 features are: total_rec_prncp, total_pymnt_inv, and total_rec_int.